**Introduction**

***Background***

 The aim of this project is to find the best location to open an Indian Restaurant in Paris.It provides an idea on what would be the best venue to start a hotel for a city where people visit from different parts of the world.It is to let people know the authentic taste of Indian food.

***Business*** ***problem*** 

The report focusses on issues to open a hotel in  city where there are already other luxury hotels available.It also puts some insight into finding a best location targeting the tourist places. 

***Data Section & Methodology***

Use the FourSquare API to collect data about restaurants.
Get coordinates of the city and create a folium map to visuaize the data.From the map we can spot the density of business.Find the hotspot and get a location to open the business at a certain distance in order to lessen the competitiveness.

***Results***

From the map,hotspot of several hotels is obtained.Using the location information find a spot at a certain distance to open the new business.


In [ ]:
#@title
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np
import folium 

In [ ]:
#@title
CLIENT_ID = '3DUTVOX4UGC3RRL5UA5MJIY4Z0ARKGYUH3TTBOZLRH1RXI40' # your Foursquare ID
CLIENT_SECRET = 'R32ZYFKBCO4YVSRFFPQJDIH2CR325OFQ3CDS1A0ADBFBD1SX' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3DUTVOX4UGC3RRL5UA5MJIY4Z0ARKGYUH3TTBOZLRH1RXI40
CLIENT_SECRET:R32ZYFKBCO4YVSRFFPQJDIH2CR325OFQ3CDS1A0ADBFBD1SX


In [ ]:
import requests

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180604',
    "section": "food",
    "near": "Paris",
    "radius": 1000,
    "limit": 50}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [ ]:
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [ ]:
d["headerLocationGranularity"], d["headerLocation"]

('city', 'Paris')

In [ ]:
d["suggestedBounds"], d["totalResults"]

({'ne': {'lat': 48.85946926480407, 'lng': 2.3577455074779436},
  'sw': {'lat': 48.84679449664059, 'lng': 2.3395638781373203}},
 247)

In [ ]:
d["geocode"]

{'cc': 'FR',
 'center': {'lat': 48.85341, 'lng': 2.3488},
 'displayString': 'Paris, France',
 'geometry': {'bounds': {'ne': {'lat': 49.03356694704047,
    'lng': 2.6042489194755207},
   'sw': {'lat': 48.670322030624696, 'lng': 2.1084049608796893}}},
 'longId': '72057594040916443',
 'slug': 'paris',
 'what': '',
 'where': 'paris'}

In [ ]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [ ]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

In [ ]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 50


{'reasons': {'count': 0,
  'items': [{'reasonName': 'globalInteractionReason',
    'summary': 'This spot is popular',
    'type': 'general'}]},
 'referralId': 'e-3-4d07738e347da1cd0c093b8f-0',
 'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/japanese_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d111941735',
    'name': 'Japanese Restaurant',
    'pluralName': 'Japanese Restaurants',
    'primary': True,
    'shortName': 'Japanese'}],
  'id': '4d07738e347da1cd0c093b8f',
  'location': {'address': "12 rue de l'Hôtel Colbert",
   'cc': 'FR',
   'city': 'Paris',
   'country': 'France',
   'formattedAddress': ["12 rue de l'Hôtel Colbert", '75005 Paris', 'France'],
   'labeledLatLngs': [{'label': 'display',
     'lat': 48.851569311075956,
     'lng': 2.3483908623771272}],
   'lat': 48.851569311075956,
   'lng': 2.3483908623771272,
   'postalCode': '75005',
   'state': 'Île-de-France'},
  'name': 'Sola',
  'photos': {'count': 0, 'groups': []}}}

In [ ]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    print(location)
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address =  ''
    if hasattr(location, 'address'):
      address = location['address']
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i hotels" % len(df))
df.head()

{'address': "12 rue de l'Hôtel Colbert", 'lat': 48.851569311075956, 'lng': 2.3483908623771272, 'labeledLatLngs': [{'label': 'display', 'lat': 48.851569311075956, 'lng': 2.3483908623771272}], 'postalCode': '75005', 'cc': 'FR', 'city': 'Paris', 'state': 'Île-de-France', 'country': 'France', 'formattedAddress': ["12 rue de l'Hôtel Colbert", '75005 Paris', 'France']}
{'address': '11 quai de Montebello', 'lat': 48.85174869628851, 'lng': 2.349318712949753, 'labeledLatLngs': [{'label': 'display', 'lat': 48.85174869628851, 'lng': 2.349318712949753}], 'postalCode': '75005', 'cc': 'FR', 'neighborhood': 'Sorbonne', 'city': 'Paris', 'state': 'Île-de-France', 'country': 'France', 'formattedAddress': ['11 quai de Montebello', '75005 Paris', 'France']}
{'address': '39 rue de la Bûcherie', 'crossStreet': 'Rue Saint-Julien le Pauvre', 'lat': 48.852637325289535, 'lng': 2.3469194769859314, 'labeledLatLngs': [{'label': 'display', 'lat': 48.852637325289535, 'lng': 2.3469194769859314}], 'postalCode': '75005

,uid,name,shortname,address,postalcode,lat,lng
0,4d07738e347da1cd0c093b8f,Sola,Japanese,,75005,48.851569,2.348391
1,508ad773e4b01ceeb2bab8d4,Comme chai Toi,French,,75005,48.851749,2.349319
2,4f3ec0bce4b0d0a64598157a,Le Petit Châtelet,French,,75005,48.852637,2.346919
3,554548c8498e731178ebf08a,Sourire Tapas Françaises,Tapas,,75005,48.851167,2.347728
4,578a2e9f498e1129b45bb0db,Shiso Burger,Burgers,,75005,48.853376,2.345302


In [ ]:
Paris_center = d["geocode"]["center"]
Paris_center

{'lat': 48.85341, 'lng': 2.3488}

In [ ]:
from folium import plugins


map_paris = folium.Map(location=[48.85341, 2.3488], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_paris)

add_markers(df)
hm_data = df[["lat", "lng"]].to_numpy().tolist()
map_paris.add_child(plugins.HeatMap(hm_data))

map_paris

In [ ]:
lat = 48.85341 
lng = 2.3488
map_paris = folium.Map(location=[lat, lng], zoom_start=17)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup='New hotel',
    color='green',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_paris)
map_paris